### 1. Fitting the parity function

In [2]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer

In [6]:
dev = qml.device('default.qubit')

TypeError: __init__() missing 1 required positional argument: 'wires'

In [4]:
def layer(layer_weights):
    for wire in range(4):
        qml.Rot(*layer_weights[wire], wires=wire)

    for wires in ([0, 1], [1, 2], [2, 3], [3, 0]):
        qml.CNOT(wires)

In [5]:
dev = qml.device('default.qubit')

def layer(layer_weights):
    for wire in range(4):
        qml.Rot(*layer_weights[wire], wires=wire)
    
    for wires in ([0,1], [1,2], [2,3], [3,0]):
        qml.CNOT(wires)

TypeError: __init__() missing 1 required positional argument: 'wires'